In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import regex
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
import os

### Hays

In [ ]:
ChristianBegriffe = ["Hadoop", "Spark", "Cloudera", "Big+Data", "Kafka", "Elastic", "Hive", "Hortonworks", "Data+Engineer", "Data+Warehouse", "Datawarehouse", "DWH", "Data+Science", "Data+Scientist", "Python", "AWS", "GCP", "Pub/Sub", "Redshift", "Lift+&+Shift", "Google+Cloud", "Dataproc", "Datafusion", "Athena", "EMR"]
MichaelBegriffe = ["Hadoop", "Big+Data", "Data+Engineer", "Data+Warehouse", "Datawarehouse", "DWH", "Business+Intelligence", "Business+Analytics", "Scrum+Master+Data", "Scrum+Master+Data+Analytics", "Data+Science", "Data+Scientist", "Machine+Learning", "Python", "Data+Vault", "Power+BI", "SSAS", "SSIS", "SSRS", "Hichert", "IBCS", "PowerQuery", "Azure+Analysis+Services", "Azure+Data+Factory", "Azure+Data+Lake", "Azure+SQL+DB", "Azure+Cognitive+Services", "Azure+ML+Services", "SQL+Services", "Data+Bricks", "MLOps", "Self-Service+BI"]

# AndreBegriffe = ["Data Engineer", "Data Warehouse", "Datawarehouse", "DWH", "Data Vault", "Talend", "Salesforce", "DataStage", "WhereScape", "PostgrSQL"]
# FrankBegriffe = ["Hadoop", "Big Data", "Hive", "Data Engineer", "Data Warehouse", "Datawarehouse", "DWH", "Datamanagement", "Data Quality", "Data governance", "Business Intelligence", "Business Analytics", "Business Analyse", "Business Analyst", "Product Owner Data", "Product Owner Data Analytics", "Scrum Master Data", "Scrum Master Data Analytics", "AI", "KI", "SAS", "Data Vault", "Self-Service BI", "ETL"]
# JannikBegriffe = ["Data Engineer", "Data Warehouse", "Datawarehouse", "DWH", "Business Intelligence", "Business Analyse", "Business Analyst", "Product Owner Data", "Product Owner Data Analytics", "Requirements Engineer Data", "Requirements Engineer Data Analytics", "Scrum Master Data", "Scrum Master Data Analytics", "Power BI", "Hichert", "IBCS", "Tableau"]
# MichaelBegriffe = ["Big Data", "Data Engineer", "Data Warehouse", "Datawarehouse", "DWH", "Business Intelligence", "Business Analytics", "Scrum Master Data", "Scrum Master Data Analystics", "Data Science", "Data Scientist", "Machine Learning", "Python", "Data Vault", "Power BI", "SSAS", "SSIS", "SSRS", "Hichert", "IBCS", "PowerQuery", "Azure Analysis Services", "Azure Data Factory", "Azure Data Lake", "Azure SQL DB", "Azure Cognitive Services", "Azure ML Services", "SQL Services", "Data Bricks", "MLOps", "Self-Service BI"]
# MichaelaBegriffe = ["SAP BW", "SAP BI", "SAP Business Warehouse", "SAP Business Objects", "SAP BO", "SAP HANA", "Lumira", "BPC"]
# SvenBegriffe = ["Business Analyse", "Business Analyst", "Scrum Master Data", "Scrum Master Data Analytics", "Data Science", "Data Scientist", "Machine Learning", "Python", "R", "R-Entwickler", "AI", "KI", "Azure Cognitive Services", "Azure ML Services", "MLOps"]
#usw.

alle_fks = [("Christian", ChristianBegriffe), ("Michael", MichaelBegriffe)] # hier auch die anderen einfügen

Tauscht Begriffe in URL aus

In [ ]:
def changeURLHays(Begriffe): 
    url_list = []
    i = 0
    url = "https://www.hays.de/jobsuche/stellenangebote-jobs/j/Contracting/3/c/Deutschland/D1641BCE-D56C-11D3-AFB2-00105AB00B48/p/1/?q=Hadoop&e=false"
    while(i < len(Begriffe) - 1):
        url = url.replace(Begriffe[i], Begriffe[i+1])
        # print(url)
        i = i+1
        url_list.append(url)
    return url_list

Speichert Ergebnislinks in Liste

In [ ]:
def getLinksHays(url): 
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    links = []
    for link in soup.findAll('a', class_ = 'search__result__header__a', attrs={'href': regex.compile("https://")}): 
        links.append(link.get('href'))
    return links
# print(getLinksHays("https://www.hays.de/jobsuche/stellenangebote-jobs/j/Contracting/3/c/Deutschland/D1641BCE-D56C-11D3-AFB2-00105AB00B48/p/1/?q=Hadoop&e=false"))

In [ ]:
def getTitleHays(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    title = soup.find('h1', class_ = 'hays__job__details__job__title').text
    return title

# print(getTitleHays("https://www.hays.de/jobsuche/stellenangebote-jobs-detail-devops-engineer-data-science-customer-relationship-management--crm-bonn-521743/1"))

In [ ]:
# diese Funktion kann auch für alle anderen Seiten aufgerufen werden (Gulp, Solcom etc.)
# Als parameter wird die String-Ergebnisliste erwartet, die dann in das File geschrieben wird,
# also: result_list = ["Führungskraft", "Jobtitel", "Suchbegriff", "aktuelle Seite (z.b.Hays)", "Link zum Job"]
# am besten in genau dieser Reihenfolge, dann kann die Führungskraft auch für den File-Namen verwendet werden
# Die Funkiton erstellt ein File "results_Führunkgsraftxy" und hängt die jeweiligen Ergebnisse immer an die bestehende Datei an.

def saveResults(result_list):
    filename = './jobboerse_{}.csv'.format(result_list[0])
    file_exists = os.path.isfile(filename)

    with open (filename, 'a+') as csvfile:
        headers = ['Für', 'Bezeichnung', 'Stichwort', 'Plattform', 'Datum', 'Link']
        writer = csv.DictWriter(csvfile, delimiter=';', lineterminator='\n',fieldnames=headers)

        if not file_exists:
            writer.writeheader()  # file doesn't exist yet, write a header

        writer.writerow({'Für': result_list[0], 'Bezeichnung': result_list[1], 
                     'Stichwort':result_list[2],'Plattform':result_list[3], 
                         'Datum':result_list[4], 'Link':result_list[5]})


In [ ]:
for fk, begriffe in alle_fks:
    print(fk)
    i = 0
    date = (datetime.date.today()).strftime('%d-%m-%Y')
    # links for keywords
    hays_urls = changeURLHays(begriffe)
    
    # get joblinks for every keyword
    for u in hays_urls:
        i += 1
        hays_job_links = getLinksHays(u)
        print(len(hays_job_links))
    
        # get title for every joblink
        for l in hays_job_links:
            title = getTitleHays(l)
            hays_result_list = [str(fk), title, str(begriffe[i]), "hays", date, l]
            saveResults(hays_result_list)
    

### Gulp

Tauscht Begriffe in URL aus

In [ ]:
def changeURLGulp(Begriffe): 
     i = 0
     url = "https://www.gulp.de/gulp2/g/projekte?scope=projects&query=Hadoop&order=DATE_DESC&region=D0&region=D1&region=D2&region=D3&region=D4&region=D5&region=D6&region=D7&region=D8&region=D9"
     while(i < len(Begriffe)): 
        url = url.replace(Begriffe[i], Begriffe[i+1])
        print(url)
        i = i+1
changeURLGulp(Begriffe)

Speichert Ergebnislinks in Liste

In [ ]:
#DEBUG
def getLinksGulp(url): 
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    links = []
    baseUrl = "https://gulp.de"
    for link in soup.findAll('a', href=True): 
        newLink = baseUrl.join(link)
        #links.append(link.get('href'))
    return newLink
print(getLinksGulp("https://www.gulp.de/gulp2/g/projekte?scope=projects&query=Hadoop&order=DATE_DESC&region=D0&region=D1&region=D2&region=D3&region=D4&region=D5&region=D6&region=D7&region=D8&region=D9"))

In [ ]:
#placeholder, not finished yet
for fk, begriffe in alle_fks:
    i = 0
    
    # links for keywords
    gulp_urls = changeURLGulp(begriffe)
    
    # get joblinks for every keyword
    for u in gulp_urls:
        i += 1
        gulp_job_links = getLinksGulp(u)
    
        # get title for every joblink
        for l in gulp_job_links:
            title = getTitleHays(l)
            gulp_result_list = [str(fk), title, str(begriffe[i]), "gulp", l]
            saveResults(gulp_result_list)
    

### Solcom

In [ ]:
# Benutze Google Chrome und es ist lokal gespeichert

driver = webdriver.Chrome('/Users/vonderheyden/Downloads/chromedriver_win32/chromedriver.exe')
driver.get('https://www.solcom.de/de/projektportal/projektangebote')

# Stellt die Standardfilter ein
def editFilter(): 
    driver.find_element_by_id("keyword").send_keys("Hadoop") # Default: Erster Begriff
    time.sleep(1)
    driver.find_element_by_class_name("prio1.acceptall").click()
    driver.find_element_by_class_name("current-selection").click()
    driver.find_element_by_xpath('//*[@id="main"]/div/div/div/div/div/div[1]/div[1]/form/div[3]/div/div[2]/ul/li[2]/div/ins').click()
    driver.find_element_by_xpath('//*[@id="main"]/div/div/div/div/div/div[1]/div[1]/form/div[3]/div/div[2]/ul/li[5]/div[1]/ins').click()
    driver.find_element_by_xpath('//*[@id="main"]/div/div/div/div/div/div[1]/div[1]/form/button[1]').click()

editFilter()



### Westhouse

In [ ]:
driver = webdriver.Chrome('/Users/vonderheyden/Downloads/chromedriver_win32/chromedriver.exe')
driver.get('https://www.westhouse-group.com/en/for_candidates')

# Stellt die Standardfilter ein
# F oder P muss bei der aktiven Informationsextraktion gefiltert werden
def editFilter(): 
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="CookieBoxSaveButton"]').click()
    time.sleep(1)
    driver.find_element_by_id('job-finder-search-keyword').send_keys("Hadoop")
    driver.find_element_by_xpath('//*[@id="headingjobtype2"]/p/a/span/i[1]').click()

editFilter()

### Freelancermap

In [ ]:
# Gefiltert ohne Hadoop
# driver.get("https://www.freelancermap.de/projektboerse.html?profisuche=1&pq_vertragsart%5Bcontract_type.remote%5D=REMOTE&pq_projekteinstellung=0&pq_sorttype=1&mCats%5B0%5D=1&mCats%5B1%5D=2&mCats%5B2%5D=3&mCats%5B3%5D=4&mCats%5B4%5D=5&mCats%5B5%5D=6&sCou%5B0%5D=1")

Begriffe = ["Hadoop", "Spark", "Cloudera", "Big+Data", "Kafka", "Elastic", "Hive", "Hortonworks", "Data+Engineer", "Data+Warehouse", "Datawarehouse", "DWH", "Data+Science", "Data+Scientist", "Python", "AWS", "GCP", "Pub/Sub", "Redshift", "Lift+%26+Shift", "Google+Cloud", "Dataproc", "Datafusion", "Athena", "EMR"]

Begriffe austauschen

In [ ]:
def changeURLFreelancer(Begriffe): 
    i = 0
    url = "https://www.freelancermap.de/?module=projekt&func=suchergebnisse&pq=Hadoop&profisuche=0&pq_sorttype=1&mCats[]=1&mCats[]=2&mCats[]=3&mCats[]=4&mCats[]=5&mCats[]=6&redirect=1"
    while(i < len(Begriffe) - 1): 
        url = url.replace(Begriffe[i], Begriffe[i+1])
        print(url)
        i = i+1
changeURLFreelancer(Begriffe)

Titel extrahieren

In [ ]:
def getTitlesFreelancer(url): 
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    titles = []
    for title in soup.findAll('li', class_ = 'project-row'): 
        titles.append(title.find('a').getText())
    titles = titles[5:]
    return titles
print(getTitlesFreelancer("https://www.freelancermap.de/?module=projekt&func=suchergebnisse&pq=Hadoop&profisuche=0&pq_sorttype=1&mCats[]=1&mCats[]=2&mCats[]=3&mCats[]=4&mCats[]=5&mCats[]=6&redirect=1"))

Links extrahieren

In [17]:
def getLinksFreelancer(url): 
    prefix = "https://www.freelancermap.de" 
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    links = []
    for link in soup.findAll('a', attrs={'href': regex.compile("/projektboerse/projekte/it/")}): 
        links.append(link.get('href'))
        #DEBUG
    for link in links: 
        link.replace(link, prefix + link)
    return links
print(getLinksFreelancer("https://www.freelancermap.de/?module=projekt&func=suchergebnisse&pq=Hadoop&profisuche=0&pq_sorttype=1&mCats[]=1&mCats[]=2&mCats[]=3&mCats[]=4&mCats[]=5&mCats[]=6&redirect=1"))

['/projektboerse/projekte/it/2057007-projekt-senior-engineer-business-functional-analyst-project-manager-solution-architect-bonn.html', '/projektboerse/projekte/it/2056590-projekt-nifi-developer-d-darmstadt.html', '/projektboerse/projekte/it/2056341-projekt-apache-nifi-developer--6-months--remote-germany--start-asap-deutschland.html', '/projektboerse/projekte/it/2056326-projekt-system-engineer-architect-m-f-d-wanted-cpat2302212-berlin.html', '/projektboerse/projekte/it/2056280-projekt-pyspark-scala-muelheim.html', '/projektboerse/projekte/it/2056275-projekt-apache-nifi-developer-english-remote-frankfurt.html', '/projektboerse/projekte/it/2056269-projekt-nifi-developer-remote-or-berlin.html', '/projektboerse/projekte/it/2056200-projekt-apache-nifi-developer-english-remote-frankfurt.html', '/projektboerse/projekte/it/2056182-projekt-pyspark-scala-lead-expert-frankfurt-germany.html', '/projektboerse/projekte/it/2056143-projekt-data-engineer-with-cloudera-experience-freelance-100-remote-ge

Projekt-ID extrahieren

In [21]:
def getProjectID(link): 
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'lxml')
    return soup.find('div', class_ = "project-detail-description")

print(getProjectID("https://www.freelancermap.de/projektboerse/projekte/it/2057007-projekt-senior-engineer-business-functional-analyst-project-manager-solution-architect-bonn.html"))

<div class="project-detail-description">
                
                                                        Remote
                            </div>
